In [44]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from graphviz import Digraph
import torch
import torch.nn.functional as F


In [65]:
words = open('names.txt', 'r').read().splitlines()

for word in words[:10]:
  print(word)

words = ["."] + [f"{word}." for word in words]
print(words)
chars = sorted(list(set(''.join(words))))
print(chars)

stoi = {s:i for i,s in enumerate(chars)}
print(stoi)
itos = {i:s for s,i in stoi.items()}
print(itos)

def build_dataset(words, block_size, is_print=False):  
  X, Y = [], []
  for word in words:
    context = [0] * block_size
    for ch in word:
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      if is_print:
        print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  # print(X.shape, Y.shape)
  return X, Y

emma
olivia
ava
isabella
sophia
charlotte
mia
amelia
harper
evelyn
['.', 'emma.', 'olivia.', 'ava.', 'isabella.', 'sophia.', 'charlotte.', 'mia.', 'amelia.', 'harper.', 'evelyn.', 'abigail.', 'emily.', 'elizabeth.', 'mila.', 'ella.', 'avery.', 'sofia.', 'camila.', 'aria.', 'scarlett.', 'victoria.', 'madison.', 'luna.', 'grace.', 'chloe.', 'penelope.', 'layla.', 'riley.', 'zoey.', 'nora.', 'lily.', 'eleanor.', 'hannah.', 'lillian.', 'addison.', 'aubrey.', 'ellie.', 'stella.', 'natalie.', 'zoe.', 'leah.', 'hazel.', 'violet.', 'aurora.', 'savannah.', 'audrey.', 'brooklyn.', 'bella.', 'claire.', 'skylar.', 'lucy.', 'paisley.', 'everly.', 'anna.', 'caroline.', 'nova.', 'genesis.', 'emilia.', 'kennedy.', 'samantha.', 'maya.', 'willow.', 'kinsley.', 'naomi.', 'aaliyah.', 'elena.', 'sarah.', 'ariana.', 'allison.', 'gabriella.', 'alice.', 'madelyn.', 'cora.', 'ruby.', 'eva.', 'serenity.', 'autumn.', 'adeline.', 'hailey.', 'gianna.', 'valentina.', 'isla.', 'eliana.', 'quinn.', 'nevaeh.', 'ivy.',

In [72]:
# train
g = torch.Generator().manual_seed(2147483647)

n_chars = 27
n_embd = 2
n_block_size = 3
n_hidden = 100
n_batch = 1000

# splits of training, dev and test
# 80%, 10%, 10%
import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1], block_size=n_block_size)
Xval, Yval = build_dataset(words[n1:n2], block_size=n_block_size)
Xte, Yte = build_dataset(words[n2:], block_size=n_block_size)

C = torch.randn((n_chars, n_embd), generator=g)
W1 = torch.randn((n_embd * n_block_size, n_hidden), generator=g)
b1 = torch.randn(n_hidden, generator=g)
W2 = torch.randn((n_hidden, n_chars), generator=g)
b2 = torch.randn(n_chars, generator=g)

parameters = [C, W1, b1, W2, b2]
for parameter in parameters:
    parameter.requires_grad = True

for i in range(10000):

    # forward pass
    ix_batch = torch.randint(0, Xtr.shape[0], (n_batch,))
    emb = C[Xtr[ix_batch]]
    emb = emb.view(emb.shape[0], -1)
    h = torch.tanh(emb @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix_batch])

    # backward pass
    for parameter in parameters:
        parameter.grad = None
    loss.backward()

    # update
    for parameter in parameters:
        parameter.data -= 0.1 * parameter.grad

    print(f"{i=} {loss.item()=}")

# train loss
emb = C[Xtr]
emb = emb.view(emb.shape[0], -1)
h = torch.tanh(emb @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
print(f"train loss: {loss.item()=}")

# validation loss
emb = C[Xval]
emb = emb.view(emb.shape[0], -1)
h = torch.tanh(emb @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Yval)
print(f"validation loss: {loss.item()=}")



i=0 loss.item()=19.29595947265625
i=1 loss.item()=16.769290924072266
i=2 loss.item()=15.25120735168457
i=3 loss.item()=14.892759323120117
i=4 loss.item()=14.17982006072998
i=5 loss.item()=13.093823432922363
i=6 loss.item()=12.757349014282227
i=7 loss.item()=11.489216804504395
i=8 loss.item()=11.467278480529785
i=9 loss.item()=11.02012825012207
i=10 loss.item()=10.689330101013184
i=11 loss.item()=11.135738372802734
i=12 loss.item()=10.394919395446777
i=13 loss.item()=10.07617473602295
i=14 loss.item()=9.845492362976074
i=15 loss.item()=9.116148948669434
i=16 loss.item()=9.174164772033691
i=17 loss.item()=9.099471092224121
i=18 loss.item()=9.004347801208496
i=19 loss.item()=8.53467082977295
i=20 loss.item()=8.537052154541016
i=21 loss.item()=8.054922103881836
i=22 loss.item()=7.910107612609863
i=23 loss.item()=7.930073261260986
i=24 loss.item()=7.3598527908325195
i=25 loss.item()=7.436681270599365
i=26 loss.item()=7.361224174499512
i=27 loss.item()=7.166617393493652
i=28 loss.item()=6.89

In [73]:
# inference
g = torch.Generator().manual_seed(2147483647)
for i in range(10):
    context = [0] * n_block_size
    out = []
    while True:

        # Forward pass
        emb = C[torch.tensor([context])]
        emb = emb.view(emb.shape[0], -1)
        h = torch.tanh(emb @ W1 + b1)
        logits = h @ W2 + b2
        p = F.softmax(logits, dim=1)

        # Sample
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()

        # Update context
        context = context[1:] + [ix]

        # Append to output
        out.append(itos[ix])
        if ix == 0:
            break

    print(''.join(out))

cexze.
zomlyurailezity.
kerinimi.
tain.
luzan.
kan.
arre.
zyauhlansr.
gotai.
molien.
